<a href="https://colab.research.google.com/github/vadigr123/colab-testing/blob/main/CivitAI_TXT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 📄 Civitai Models + TXT File
import requests, re, os, time
from IPython.display import clear_output

# 🔗 Enter a valid Civitai user profile URL:
user_url = "https://civitai.com/user/vadigr123_"  #@param {type:"string"}

# 🎯 Extract username
match = re.search(r'/user/([^/?#]+)', user_url)
if not match:
    raise ValueError("❌ Invalid user URL!")
username = match.group(1)

models = []
page = 1
print(f"🔍 Fetching models for user: {username}...\n")

# 🧠 Load all model pages from API
while True:
    url = f"https://civitai.com/api/v1/models?username={username}&limit=100&page={page}"
    try:
        res = requests.get(url)
        data = res.json()
    except Exception as e:
        print(f"❌ Failed to load page {page}: {e}")
        break

    if not data.get("items"):
        break

    models.extend(data["items"])
    page += 1

if not models:
    raise RuntimeError("❌ No models found!")

total = len(models)
log_lines = []
output_lines = []

# 📦 Build the list
for idx, model in enumerate(models, start=1):
    clear_output(wait=True)
    model_name = model["name"]
    model_line = f"{idx}/{total} [{round((idx/total)*100)}%] 📦 {model_name}"
    print(model_line)
    success = True
    version_lines = []

    try:
        for version in model.get("modelVersions", []):
            ver_name = version["name"]
            ver_id = version["id"]
            ver_url = f"https://civitai.com/api/download/models/{ver_id}"
            version_lines.append(f"{ver_name} {ver_url}")
            log_lines.append(f"✅ Downloaded: {model_name} | {ver_name}")
    except Exception as e:
        log_lines.append(f"❌ Skipped: {model_name} | Error: {str(e)}")
        success = False

    if success:
        output_lines.append(model_name)
        output_lines.extend(version_lines)
        output_lines.append("")  # spacing between models

    # Show recent log lines (last 5)
    print("\n".join(log_lines[-5:]))

# ✍️ Save to .txt file
filename = f"{username}_models.txt"
with open(filename, "w", encoding="utf-8") as f:
    f.write("\n".join(output_lines))

# ✅ Final output
clear_output(wait=True)
print("✅ All done!\n")
for line in log_lines:
    print(line)

print(f"\n📄 TXT file saved as: `{filename}` ({len(models)} models listed)")
